# Tree with state propagation

In [1]:
%matplotlib widget
import bmcs_utils.api as bu
import traits.api as tr
from bmcs_utils.model import Model

In [2]:
scd = False

Each model can be regarded as a tree of model components.
Let us define models `TopModel`, `InterimModel` and `SubModel`.

In [3]:
class SubModel(Model):
    name = 'submodel'
    
    length = bu.Float(2, GEO=True)

    ipw_view = bu.View(
        bu.Item('length')
    )

In [4]:
sm = SubModel()
sm.state_change_debug = True

In [5]:
sm.length = 3

value_changed <__main__.SubModel object at 0x7fc90964fb80> TraitChangeEvent(object=<__main__.SubModel object at 0x7fc90964fb80>, name='length', old=2, new=3.0)


Let us define a more complex model

In [6]:
class InterimModel(Model):
    sm = bu.Instance(SubModel,())
    
    stiffness = tr.Property(bu.Float, depends_on='state_changed')
    @tr.cached_property
    def _get_stiffness(self):
        return self.sm.length * 10
    
    depends_on = ['sm']
    
    ipw_view = bu.View(
        bu.Item('stiffness', readonly=True)
    )

In [7]:
im = InterimModel(sm=sm)
im.state_change_debug = True

In [8]:
im.sm.length = 8

value_changed <__main__.SubModel object at 0x7fc90964fb80> TraitChangeEvent(object=<__main__.SubModel object at 0x7fc90964fb80>, name='length', old=3.0, new=8.0)
value_changed <__main__.InterimModel object at 0x7fc909808ae0> Notification from child <__main__.SubModel object at 0x7fc90964fb80>


In [9]:
class TopLevelModelEagerGraphChange(Model):
    name = 'tm'
    
    sm = bu.Instance(SubModel, ())
    def _sm_default(self):
        sm = SubModel()
        self.set_sm(sm)
        return sm
    def _sm_changed(self):
        self.set_sm(self.sm)
    def set_sm(self, sm):
        self.im.sm = sm
        self.im2.sm = sm
        
    im = bu.Instance(InterimModel, ())
    im2 = bu.Instance(InterimModel, ())

    time = bu.Float(400, ALG=True)

    tree = ['sm', 'im', 'im2']


In [10]:
tl = TopLevelModelEagerGraphChange()
tl.state_change_debug = True

In [11]:
tl.sm = SubModel()
tl.sm, tl.im2.sm, tl.im.sm

parent <__main__.InterimModel object at 0x7fc909544ef0> changing child from child <__main__.SubModel object at 0x7fc909dbad60> to <__main__.SubModel object at 0x7fc93c132c20>
graph_changed <__main__.InterimModel object at 0x7fc909544ef0> Notification from child <__main__.SubModel object at 0x7fc93c132c20>
graph_changed <__main__.TopLevelModelEagerGraphChange object at 0x7fc90985e9f0> Notification from child <__main__.InterimModel object at 0x7fc909544ef0>
parent <__main__.InterimModel object at 0x7fc90954e6d0> changing child from child <__main__.SubModel object at 0x7fc909dbad60> to <__main__.SubModel object at 0x7fc93c132c20>
graph_changed <__main__.InterimModel object at 0x7fc90954e6d0> Notification from child <__main__.SubModel object at 0x7fc93c132c20>
graph_changed <__main__.TopLevelModelEagerGraphChange object at 0x7fc90985e9f0> Notification from child <__main__.InterimModel object at 0x7fc90954e6d0>
parent <__main__.TopLevelModelEagerGraphChange object at 0x7fc90985e9f0> changin

(<__main__.SubModel at 0x7fc93c132c20>,
 <__main__.SubModel at 0x7fc93c132c20>)

In [12]:
tl.im.parents, tl.im2.parents, tl.sm.parents

({<__main__.TopLevelModelEagerGraphChange at 0x7fc90985e9f0>},
 {<__main__.TopLevelModelEagerGraphChange at 0x7fc90985e9f0>},
 {<__main__.InterimModel at 0x7fc909544ef0>,
  <__main__.TopLevelModelEagerGraphChange at 0x7fc90985e9f0>})

In [13]:
tl.im.sm.length = 10

value_changed <__main__.SubModel object at 0x7fc93c132c20> TraitChangeEvent(object=<__main__.SubModel object at 0x7fc93c132c20>, name='length', old=2, new=10.0)
value_changed <__main__.TopLevelModelEagerGraphChange object at 0x7fc90985e9f0> Notification from child <__main__.SubModel object at 0x7fc93c132c20>
value_changed <__main__.InterimModel object at 0x7fc90954e6d0> Notification from child <__main__.SubModel object at 0x7fc93c132c20>
value_changed <__main__.TopLevelModelEagerGraphChange object at 0x7fc90985e9f0> Notification from child <__main__.InterimModel object at 0x7fc90954e6d0>
value_changed <__main__.InterimModel object at 0x7fc909544ef0> Notification from child <__main__.SubModel object at 0x7fc93c132c20>
value_changed <__main__.TopLevelModelEagerGraphChange object at 0x7fc90985e9f0> Notification from child <__main__.InterimModel object at 0x7fc909544ef0>


In [14]:
tl.im, tl.im2, tl.im.sm.parents

(<__main__.InterimModel at 0x7fc909544ef0>,
 {<__main__.InterimModel at 0x7fc909544ef0>,
  <__main__.TopLevelModelEagerGraphChange at 0x7fc90985e9f0>})

In [15]:
tl.interact()

In [16]:
im.sm.length

8.0

In [17]:
class TopLevelModelLazyGraphChange(Model):
    name = 'tm'
    
    sm = bu.Instance(SubModel, ())
        
    im = tr.Property(bu.Instance(InterimModel), depends_on='sm')
    @tr.cached_property
    def _get_im(self):
        print('im')
        return InterimModel(sm=self.sm)

    im2 = tr.Property(bu.Instance(InterimModel), depends_on='sm')
    @tr.cached_property
    def _get_im2(self):
        print('im2')
        return InterimModel(sm=self.sm)

    time = bu.Float(400, ALG=True)

    tree = ['sm', 'im', 'im2']

In [18]:
tl = TopLevelModelEagerGraphChange()
tl.sm.parents, tl.children_traits, tl.im.children_traits

({<__main__.InterimModel at 0x7fc9013f5770>,
  <__main__.TopLevelModelEagerGraphChange at 0x7fc9013df810>},
 {'sm': <__main__.SubModel at 0x7fc901483c70>,
  'im': <__main__.InterimModel at 0x7fc901491ef0>,
  'im2': <__main__.InterimModel at 0x7fc9013f5770>},
 {'sm': <__main__.SubModel at 0x7fc9013edb30>})

In [19]:
tl.interact()

Add change the component and check if the changes of stiffness in InterimModel get updated

In [21]:
tl.sm = SubModel()
tl.sm.length = 30
tl.im.stiffness

300.0